## Screenering reports

Below is an example of how you can quickly filter a corpus of PFD reports using natural language.

While the judiciary.uk website does offer a search function, it relies solely on keyword matching. Because coroners often refer to the same core concepts in different ways, this approach can miss relevant reports. Additionally, the judiciary.uk search only works for newer reports with embedded HTML; older reports are excluded, making comprehensive searching difficult. 

PFD Toolkit's Screener module offers greater flexibility and ensures access to the full archive.

First, we are going to load a small sample of reports to work from. While we're here, we'll also import the required `LLM` and `Screener` modules.

In [1]:
%%time

from pfd_toolkit import load_reports, LLM, Screener
from dotenv import load_dotenv 
import os

reports = load_reports(start_date="2025-04-01", end_date="2025-05-25")
len(reports)

CPU times: user 2.31 s, sys: 117 ms, total: 2.43 s
Wall time: 773 ms


76

Next, we need to load our OpenAI API key and initialise our LLM client engine.

In [2]:
%%time

# Load OpenAI API key
load_dotenv("api.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialise LLM client
llm_client = LLM(api_key=openai_api_key, max_workers=30, model="gpt-4o-mini")

CPU times: user 29.1 ms, sys: 7.82 ms, total: 36.9 ms
Wall time: 35.7 ms


Now, all we need to do is run our `screen_reports()` method, and it will comb through the provided catelogue, pulling out reports which meet the criteria.

Below, we're going to use a simple query: "suicide".

In [3]:
%%time

filter_object = Screener(llm=llm_client,
                        reports = reports,
                        user_query="suicide")

filtered_reports = filter_object.screen_reports().reset_index(drop=True)
filtered_reports

Sending requests to the LLM: 100%|██████████| 76/76 [00:11<00:00,  6.83it/s]

CPU times: user 1.42 s, sys: 98.6 ms, total: 1.52 s
Wall time: 11.3 s


,URL,ID,Date,CoronerName,Area,Receiver,InvestigationAndInquest,CircumstancesOfDeath,MattersOfConcern
0,https://www.judiciary.uk/prevention-of-future-...,2025-0240,2025-05-21,Andrew Morse,South Wales Central,The Chief Executive Cardiff & Vale University ...,On 30 October 2023 I commenced an investigatio...,These were recorded as follows Robert Maxwell ...,During the course of the inquest the evidence ...
1,https://www.judiciary.uk/prevention-of-future-...,2025-0235,2025-05-20,Louise Hunt,Birmingham and Solihull,West Midlands Fire Service 1,On 5 February 2024 I commenced an investigatio...,Mr Brown was found hanging [REDACTED] at his h...,During the course of the inquest the evidence ...
2,https://www.judiciary.uk/prevention-of-future-...,2025-0234,2025-05-17,Sarah Murphy,Cheshire,The Royal College of General Practitioners (RC...,On 09 September 2024 I commenced an investigat...,The deceased was 28 years of age with a medica...,During the course of the investigation my inqu...
3,https://www.judiciary.uk/prevention-of-future-...,2025-0227,2025-05-13,Joanne Andrews,"West Sussex, Brighton and Hove",1 Sussex Partnership NHS Foundation Trust 2 NH...,On 08 March 2023 I commenced an investigation ...,Margaret Kagure Pauline Reece attended her GP ...,During the course of the investigation my inqu...
4,https://www.judiciary.uk/prevention-of-future-...,2025-0229,2025-05-08,Roland Wooderson,Gloucestershire,Department of Health and Social Care The Chief...,On 30 June 2023 I commenced an investigation i...,The deceased had a history of mental health di...,During the course of the inquest the evidence ...
5,https://www.judiciary.uk/prevention-of-future-...,2025-0211,2025-05-02,Victoria Davies,Cheshire,1 Governor for HMP Styal 2 Minister of State f...,On 24 July 2024 I commenced an investigation i...,Sarah Boyle was detained at HMP Styal between ...,During the course of the investigation my inqu...
6,https://www.judiciary.uk/prevention-of-future-...,2025-0120,2025-04-24,Samantha Marsh,Somerset,Part 1 1. Somerset Foundation Trust of Trust M...,On 6 th December 2022 I commenced an investiga...,Anne first presented to her GP in 2008. During...,4 During the course of the inquest the evidenc...
7,https://www.judiciary.uk/prevention-of-future-...,2025-0120,2025-04-17,Sean Horstead,Essex,Essex County Council,On 24 th November 2023 I commenced an investig...,Linda's husband had been suffering from signif...,During the inquest the evidence revealed matte...
8,https://www.judiciary.uk/prevention-of-future-...,2025-0184,2025-04-10,Lydia Brown,West London,"1. Chief Executive, South West London and St G...",On 30 August I opened 2023 an investigation in...,Jonathan had been under the care of mental hea...,During the course of the inquest the evidence ...
9,https://www.judiciary.uk/prevention-of-future-...,2025-0167,2025-04-03,Kate Bisset,Lancashire and Blackburn with Darwen,NHS Pathways,On the 3 rd April 2024 the Coroner's Office wa...,Mr James Masheter was a 42-year-old man with a...,During the course of the inquest the evidence ...


The result is a smaller dataframe, containing reports which meet the user query.

### Further options

In this example, we're filtering by a broader (and wordier!) query. We want to pull out reports which mention some form of medication.

We've also customised our output through a series of additional parameters...
* `include_*` - we can toggle any PFD reports section on or off. If our query relates to one specific section (Circumstances of Death), then we don't really need the model to see the Investigation or Concerns sections. By default, the shorter sections are also turned off (i.e. receiver, coroner name, area) but you can manually turn these on too.
* `match_leniency` - either *strict* or *liberal* (defaults to *strict*). If the AI is on the fence whether a report is relevant to your query, you can decide to exclude it (strict) or include it (liberal) in the final dataframe.
* `filter_df` - either *True* or *False* (defaults to *True*). Whether to filter the dataframe so that only relevant reports are returned (*True*), or whether to return a column which indicates whether the report fulfils the query or not.
* `result_col_name` - if above is set to False, you can optionally denote the column name you want added to your dataframe.

In [4]:
%%time

user_query = "Reports which explicitly mention death from a specific medication."

advanced_filter = Screener(reports=reports, 
                         llm=llm_client,
                         user_query=user_query, 
                        
                         include_circumstances=True,
                         include_investigation=False,
                         include_concerns=False,
                         
                         match_leniency='liberal', 
                         filter_df=False,
                         result_col_name='medication_error')

medication_reports = advanced_filter.screen_reports().reset_index(drop=True)
medication_reports


Sending requests to the LLM (in parallel): 100%|██████████| 76/76 [00:01<00:00, 41.02it/s]

CPU times: user 813 ms, sys: 73.5 ms, total: 886 ms
Wall time: 2.11 s


,URL,ID,Date,CoronerName,Area,Receiver,InvestigationAndInquest,CircumstancesOfDeath,MattersOfConcern,medication_error
0,https://www.judiciary.uk/prevention-of-future-...,2025-0241,2025-05-23,Mary Hassell,Inner North London,1. Commissioner Metropolitan Police Service (M...,"On 12 February 2016, I commenced an investigat...",Lewis Johnson died as a consequence of a road ...,"2 During the course of the inquest, the eviden...",False
1,https://www.judiciary.uk/prevention-of-future-...,2025-0247,2025-05-23,Nadia Persaud,East London,"1. , CEO, North East London Foundation Trust (...",On 27 November 2024 I commenced an investigati...,Mr. Fraser was a 37-year-old gentleman who had...,During the course of the inquest the evidence ...,False
2,https://www.judiciary.uk/prevention-of-future-...,2025-0242,2025-05-23,Mary Hassell,Inner North London,1. Director General Independent Office for Pol...,"On 12 February 2016, I commenced an investigat...",Lewis Johnson died as a consequence of a road ...,"2 During the course of the inquest, the eviden...",False
3,https://www.judiciary.uk/prevention-of-future-...,2025-0236,2025-05-21,Kate Robertson,North West Wales,Betsi Cadwaladr University Health Board (BCUHB) 1,On 20 May 2024 I commenced an investigation in...,The circumstances of the death are as follows ...,"During the course of the inquest, the evidence...",False
4,https://www.judiciary.uk/prevention-of-future-...,2025-0239,2025-05-21,John Thompson,Northumberland,NHS England,On 17th July 2024 I commenced an investigation...,Mr Morris was diagnosed with penile cancer in ...,N/A: Not found,False
...,...,...,...,...,...,...,...,...,...,...
71,https://www.judiciary.uk/prevention-of-future-...,2025-0210,2025-04-03,Alison Hewitt,City of London,"1. , Company Secretary of Samba London Limited...",I have commenced an investigation into the dea...,"On the evidence currently available to me, I u...",The evidence I have gathered to date reveals m...,False
72,https://www.judiciary.uk/prevention-of-future-...,2025-0167,2025-04-03,Kate Bisset,Lancashire and Blackburn with Darwen,NHS Pathways,On the 3 rd April 2024 the Coroner's Office wa...,Mr James Masheter was a 42-year-old man with a...,During the course of the inquest the evidence ...,False
73,https://www.judiciary.uk/prevention-of-future-...,2025-0174,2025-04-03,Guy Davies,Cornwall and the Isles of Scilly,", Secretary of State for Health and Social Care 1",On 30 May 2024 I commenced an investigation in...,1. The findings of fact on how Andrew died are...,During the course of the inquest the evidence ...,False
74,https://www.judiciary.uk/prevention-of-future-...,2025-0178,2025-04-03,Crispin Oliver,County Durham and Darlington,REDACTED,On 26/05/2023 14:31an investigation was commen...,"Died at [REDACTED], Darlington, on 13 May 2023...",During the course of the investigation my inqu...,False
